# MNIST trained with MobileNet v2

### Links
- how to convert MNIST from 1 channel to 3 channels
  - https://discuss.pytorch.org/t/best-way-to-deal-with-1-channel-images/26699
  - Lambda transform: 
- some person's code - https://github.com/marvis/pytorch-mobilenet/blob/master/main.py
- another person's code - https://github.com/tonylins/pytorch-mobilenet-v2/blob/master/MobileNetV2.py


In [1]:
import sys
sys.path.append("./pytorch-mobilenet-v2/")
from MobileNetV2 import MobileNetV2

In [2]:
import argparse

# from kindling.nan_police import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
print(torch.__version__)
print(torch.__file__)

1.0.1.post2
/Users/william/.local/share/virtualenvs/william-oBc2a6gD/lib/python3.7/site-packages/torch/__init__.py


In [4]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.0001, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
parser.add_argument('--save-model', action='store_true', default=False,
                    help='For Saving the current Model')


_StoreTrueAction(option_strings=['--save-model'], dest='save_model', nargs=0, const=True, default=False, type=None, choices=None, help='For Saving the current Model', metavar=None)

In [5]:
args = parser.parse_args("")
args

Namespace(batch_size=64, epochs=10, log_interval=10, lr=0.0001, momentum=0.5, no_cuda=False, save_model=False, seed=1, test_batch_size=1000)

In [6]:
use_cuda = not args.no_cuda and torch.cuda.is_available()
use_cuda

False

In [7]:
torch.manual_seed(args.seed)

In [8]:
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cpu')

In [9]:
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
kwargs

{}

In [19]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.Resize((32, 32), interpolation=2),
                       transforms.Lambda(lambda image: image.convert('RGB')),
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)


In [20]:
model = MobileNetV2(n_class=10, input_size=32, width_mult=1.).to(device)
model

MobileNetV2(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace)
        (3): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace)
        (3): Conv2d(96

In [21]:
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.0001
    momentum: 0.5
    nesterov: False
    weight_decay: 0
)

In [22]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
#         print(data.shape, model)
        data, target = data.to(device), target.to(device)  # to do: needed?
        optimizer.zero_grad()  # what does this do? zero-out any previous gradient info?
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [ ]:
for epoch in range(100000):
    train(args, model, device, train_loader, optimizer, epoch=epoch)

Train Epoch: 0 [0/60000 (0%)]	Loss: -0.063519
Train Epoch: 0 [640/60000 (1%)]	Loss: -0.107796
Train Epoch: 0 [1280/60000 (2%)]	Loss: -0.125203
Train Epoch: 0 [1920/60000 (3%)]	Loss: -0.149229
Train Epoch: 0 [2560/60000 (4%)]	Loss: -0.215716
Train Epoch: 0 [3200/60000 (5%)]	Loss: -0.281049
Train Epoch: 0 [3840/60000 (6%)]	Loss: -0.304628
Train Epoch: 0 [4480/60000 (7%)]	Loss: -0.343267
Train Epoch: 0 [5120/60000 (9%)]	Loss: -0.439164
Train Epoch: 0 [5760/60000 (10%)]	Loss: -0.484212
Train Epoch: 0 [6400/60000 (11%)]	Loss: -0.436951
Train Epoch: 0 [7040/60000 (12%)]	Loss: -0.548351
Train Epoch: 0 [7680/60000 (13%)]	Loss: -0.504413
Train Epoch: 0 [8320/60000 (14%)]	Loss: -0.613533
Train Epoch: 0 [8960/60000 (15%)]	Loss: -0.630190
Train Epoch: 0 [9600/60000 (16%)]	Loss: -0.738911
Train Epoch: 0 [10240/60000 (17%)]	Loss: -0.742677
Train Epoch: 0 [10880/60000 (18%)]	Loss: -0.755252
Train Epoch: 0 [11520/60000 (19%)]	Loss: -0.796173
Train Epoch: 0 [12160/60000 (20%)]	Loss: -0.860474
Train Epo

Train Epoch: 1 [43520/60000 (72%)]	Loss: -7.645267
Train Epoch: 1 [44160/60000 (74%)]	Loss: -7.543739
Train Epoch: 1 [44800/60000 (75%)]	Loss: -7.599842
Train Epoch: 1 [45440/60000 (76%)]	Loss: -7.533133
Train Epoch: 1 [46080/60000 (77%)]	Loss: -7.859942
Train Epoch: 1 [46720/60000 (78%)]	Loss: -7.513257
Train Epoch: 1 [47360/60000 (79%)]	Loss: -7.716828
Train Epoch: 1 [48000/60000 (80%)]	Loss: -8.051358
Train Epoch: 1 [48640/60000 (81%)]	Loss: -8.107587
Train Epoch: 1 [49280/60000 (82%)]	Loss: -7.866149
Train Epoch: 1 [49920/60000 (83%)]	Loss: -8.120288
Train Epoch: 1 [50560/60000 (84%)]	Loss: -8.065267
Train Epoch: 1 [51200/60000 (85%)]	Loss: -8.232064
Train Epoch: 1 [51840/60000 (86%)]	Loss: -8.143577
Train Epoch: 1 [52480/60000 (87%)]	Loss: -8.665053
Train Epoch: 1 [53120/60000 (88%)]	Loss: -8.332055
Train Epoch: 1 [53760/60000 (90%)]	Loss: -8.754809
Train Epoch: 1 [54400/60000 (91%)]	Loss: -8.708782
Train Epoch: 1 [55040/60000 (92%)]	Loss: -8.493017
Train Epoch: 1 [55680/60000 (93

Train Epoch: 3 [25600/60000 (43%)]	Loss: -25.735809
Train Epoch: 3 [26240/60000 (44%)]	Loss: -23.342802
Train Epoch: 3 [26880/60000 (45%)]	Loss: -22.978359
Train Epoch: 3 [27520/60000 (46%)]	Loss: -21.592426
Train Epoch: 3 [28160/60000 (47%)]	Loss: -21.961998
Train Epoch: 3 [28800/60000 (48%)]	Loss: -23.849142
Train Epoch: 3 [29440/60000 (49%)]	Loss: -24.300022
Train Epoch: 3 [30080/60000 (50%)]	Loss: -25.304148
Train Epoch: 3 [30720/60000 (51%)]	Loss: -21.041660
Train Epoch: 3 [31360/60000 (52%)]	Loss: -23.368267
Train Epoch: 3 [32000/60000 (53%)]	Loss: -24.533176
Train Epoch: 3 [32640/60000 (54%)]	Loss: -25.634233
Train Epoch: 3 [33280/60000 (55%)]	Loss: -26.990885
Train Epoch: 3 [33920/60000 (57%)]	Loss: -25.401953
Train Epoch: 3 [34560/60000 (58%)]	Loss: -26.875601


In [ ]:
torch.optim.Optimizer

In [ ]:
type(optimizer)

In [ ]:
isinstance(optimizer, torch.optim.SGD)

In [ ]:
issubclass(type(optimizer), torch.optim.Optimizer)